<h1>Ensaio para Active Voxel</h1>

In [56]:
# quads always start with 6 quads needed to form the seed
# <(( int e1, int e2 ) t1, ( int e3, int e4 ) t2 ), int >
quads = {
    # front
    (( 1, 2 ), ( 3, 4 )): 0,

    # top
    (( -3, 5 ), ( 6, 7 )): 1,

    # back
    (( -6, 8 ), ( 9, 10 )): 2,

    # bottom
    (( -9, 11 ), ( -1, 12 )): 3,

    # left
    (( -11, -8 ), ( -5, -2 )): 4,

    # right
    (( -12, -4 ), ( -7, -10 )): 5
}

print( quads )

{((1, 2), (3, 4)): 0, ((-3, 5), (6, 7)): 1, ((-6, 8), (9, 10)): 2, ((-9, 11), (-1, 12)): 3, ((-11, -8), (-5, -2)): 4, ((-12, -4), (-7, -10)): 5}


In [2]:
quads_rev = {}

# dict comprehension failed here
for quad in quads:
    quads_rev[ quads[ quad ]] = quad
    
print( quads_rev )

{0: ((1, 2), (3, 4)), 1: ((-3, 5), (6, 7)), 2: ((-6, 8), (9, 10)), 3: ((-9, 11), (-1, 12)), 4: ((-11, -8), (-5, -2)), 5: ((-12, -4), (-7, -10))}


In [4]:
edges_quads = {
    1: ( 0, 3 ),  2: ( 0, 4 ),  3: ( 0, 1 ),  4: ( 0, 5 ),
    5: ( 1, 4 ),  6: ( 1, 2 ),  7: ( 1, 5 ),  8: ( 2, 4 ),
    9: ( 2, 3 ), 10: ( 2, 5 ), 11: ( 3, 4 ), 12: ( 3, 5 )
}

print( edges_quads )

{1: (0, 3), 2: (0, 4), 3: (0, 1), 4: (0, 5), 5: (1, 4), 6: (1, 2), 7: (1, 5), 8: (2, 4), 9: (2, 3), 10: (2, 5), 11: (3, 4), 12: (3, 5)}


In [5]:
scale = 1.0

vertices = {
    # front
    # edge 1 = vx 1 -> 2
    1: ( scale, 0, 0, False ),     # vx 1
    2: ( 0, 0, 0, False ),         # vx 2

    # edge 2 = vx 2 (shared edge 1) -> 3
    3: ( 0, scale, 0, False ),     # vx 3

    # edge 3 = vx 3 (shared edge 2) -> 4
    4: ( scale, scale, 0, False ), # vx 4

    # edge 4 = vx 4 (shared edge 3) -> vx1 (shared edge 1) 

    # top
    # -1 * edge 3

    # edge 5 = vx 3 (shared edge 2) -> 5
    5: ( 0, scale, scale, False ), # vx 5

    # edge 6 = vx 5 (shared edge 5) -> 6
    6: ( scale, scale, scale, False ),# vx 6

    # edge 7 = vx 6 (shared edge 6) -> vx 4 (shared edge 3)

    # back
    # -1 * edge 6

    # edge 8 = vx 5 (shared edge 5) -> 7
    7: ( 0, 0, scale, False ),     # vx 7

    # edge 9 = vx 7 (shared edge 8) -> 8
    8: ( scale, 0, scale, False ) # vx 8

    # edge 10 = vx 8 (shared edge 9) -> vx 6 (shared edge 6)

    # bottom
    # -1 * edge 9
    # edge 11 = vx 7 (shared edge 8 ) -> vx 2 (shared edge 1 )
    # -1 * edge 1
    # edge 12 = vx 1 (shared edge 1 ) -> vx 8 (shared edge 9 )

    # left
    # -1 * edge 11, -1 * edge 8, -1 * edge 5, -1 * edge 2

    # right
    # -1 * edge 12, -1 * edge 4, -1 * edge 7, -1 * edge 10
}

print( vertices )

{1: (1.0, 0, 0, False), 2: (0, 0, 0, False), 3: (0, 1.0, 0, False), 4: (1.0, 1.0, 0, False), 5: (0, 1.0, 1.0, False), 6: (1.0, 1.0, 1.0, False), 7: (0, 0, 1.0, False), 8: (1.0, 0, 1.0, False)}


In [66]:
# < int, List<( int edge, int t, int pos )>>

parent_edges = {
    1: [{ "edge": 1, "t": 1, "pos": 0 }, { "edge":  4, "t": 1, "pos": 1 }, { "edge": 12, "t": 1, "pos": 0 }],
    2: [{ "edge": 1, "t": 1, "pos": 1 }, { "edge":  2, "t": 1, "pos": 0 }, { "edge": 11, "t": 1, "pos": 1 }],
    3: [{ "edge": 2, "t": 1, "pos": 1 }, { "edge":  3, "t": 1, "pos": 0 }, { "edge":  5, "t": 1, "pos": 0 }],
    4: [{ "edge": 3, "t": 1, "pos": 1 }, { "edge":  4, "t": 1, "pos": 0 }, { "edge":  7, "t": 1, "pos": 1 }],
    5: [{ "edge": 5, "t": 1, "pos": 1 }, { "edge":  6, "t": 1, "pos": 0 }, { "edge":  8, "t": 1, "pos": 0 }],
    6: [{ "edge": 6, "t": 1, "pos": 1 }, { "edge":  7, "t": 1, "pos": 0 }, { "edge": 10, "t": 1, "pos": 1 }],
    7: [{ "edge": 8, "t": 1, "pos": 1 }, { "edge":  9, "t": 1, "pos": 0 }, { "edge": 11, "t": 1, "pos": 0 }],
    8: [{ "edge": 9, "t": 1, "pos": 1 }, { "edge": 10, "t": 1, "pos": 0 }, { "edge": 12, "t": 1, "pos": 1 }]
}

print( parent_edges )

{1: [{'edge': 1, 't': 1, 'pos': 0}, {'edge': 4, 't': 1, 'pos': 1}, {'edge': 12, 't': 1, 'pos': 0}], 2: [{'edge': 1, 't': 1, 'pos': 1}, {'edge': 2, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 1}], 3: [{'edge': 2, 't': 1, 'pos': 1}, {'edge': 3, 't': 1, 'pos': 0}, {'edge': 5, 't': 1, 'pos': 0}], 4: [{'edge': 3, 't': 1, 'pos': 1}, {'edge': 4, 't': 1, 'pos': 0}, {'edge': 7, 't': 1, 'pos': 1}], 5: [{'edge': 5, 't': 1, 'pos': 1}, {'edge': 6, 't': 1, 'pos': 0}, {'edge': 8, 't': 1, 'pos': 0}], 6: [{'edge': 6, 't': 1, 'pos': 1}, {'edge': 7, 't': 1, 'pos': 0}, {'edge': 10, 't': 1, 'pos': 1}], 7: [{'edge': 8, 't': 1, 'pos': 1}, {'edge': 9, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 0}], 8: [{'edge': 9, 't': 1, 'pos': 1}, {'edge': 10, 't': 1, 'pos': 0}, {'edge': 12, 't': 1, 'pos': 1}]}


In [7]:
# <( int edge, int t, int pos ), int >

vertex_in_edge = {}

for vx in parent_edges:
    for edge in parent_edges[ vx ]:
        vertex_in_edge[ str( edge )] = vx
    
print( vertex_in_edge )


{"{'edge': 1, 't': 1, 'pos': 0}": 1, "{'edge': 4, 't': 1, 'pos': 1}": 1, "{'edge': 12, 't': 1, 'pos': 0}": 1, "{'edge': 1, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 0}": 2, "{'edge': 11, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 1}": 3, "{'edge': 3, 't': 1, 'pos': 0}": 3, "{'edge': 5, 't': 1, 'pos': 0}": 3, "{'edge': 3, 't': 1, 'pos': 1}": 4, "{'edge': 4, 't': 1, 'pos': 0}": 4, "{'edge': 7, 't': 1, 'pos': 1}": 4, "{'edge': 5, 't': 1, 'pos': 1}": 5, "{'edge': 6, 't': 1, 'pos': 0}": 5, "{'edge': 8, 't': 1, 'pos': 0}": 5, "{'edge': 6, 't': 1, 'pos': 1}": 6, "{'edge': 7, 't': 1, 'pos': 0}": 6, "{'edge': 10, 't': 1, 'pos': 1}": 6, "{'edge': 8, 't': 1, 'pos': 1}": 7, "{'edge': 9, 't': 1, 'pos': 0}": 7, "{'edge': 11, 't': 1, 'pos': 0}": 7, "{'edge': 9, 't': 1, 'pos': 1}": 8, "{'edge': 10, 't': 1, 'pos': 0}": 8, "{'edge': 12, 't': 1, 'pos': 1}": 8}


In [68]:
# AvxEdges always starts with the 12 edges needed to build the seed
edges = {
    # front
    # edge 1 = vx 1 -> 2
    1:  { "start": 1, "end": 2, "parent": 1 },

    # edge 2 = vx 2 (shared edge 1) -> 3
    2: { "start": 2, "end": 3, "parent": 2 },

    # edge 3 = vx 3 (shared edge 2) -> 4
    3: { "start": 3, "end": 4, "parent": 3 },

    # edge 4 = vx 4 (shared edge 3) -> vx1 (shared edge 1) 
    4: { "start": 4, "end": 1, "parent": 4 },

    # top
    # edge 5 = vx 3 (shared edge 2) -> 5
    5: { "start": 3, "end": 5, "parent": 5 },

    # edge 6 = vx 5 (shared edge 5) -> 6
    6: { "start": 5, "end": 6, "parent": 6 },

    # edge 7 = vx 6 (shared edge 6) -> vx 4 (shared edge 3)
    7: { "start": 6, "end": 4, "parent": 7 },

    # back
    # edge 8 = vx 5 (shared edge 5) -> 7
    8: { "start": 5, "end": 7, "parent": 8 }, 

    # edge 9 = vx 7 (shared edge 8) -> 8
    9: { "start": 7, "end": 8, "parent": 9 },

    # edge 10 = vx 8 (shared edge 9) -> vx 6 (shared edge 6)
    10: { "start": 8, "end": 6, "parent": 10 },

    # bottom
    # edge 11 = vx 7 (shared edge 8 ) -> vx 2 (shared edge 1 )
    11: { "start": 7, "end": 2, "parent": 11 },

    # edge 12 = vx 1 (shared edge 1 ) -> vx 8 (shared edge 9 )
    12: { "start": 1, "end": 8, "parent": 12 }

    # left
    # -1 * edge 11, -1 * edge 8, -1 * edge 5, -1 * edge 2

    # right
    # -1 * edge 12, -1 * edge 4, -1 * edge 7, -1 * edge 2
}

edges_rev = {}

for _edge in edges:
    edges_rev[ str( edges[ _edge ]) ] = _edge 

    
print( edges )
print( edges_rev )


{1: {'start': 1, 'end': 2, 'parent': 1}, 2: {'start': 2, 'end': 3, 'parent': 2}, 3: {'start': 3, 'end': 4, 'parent': 3}, 4: {'start': 4, 'end': 1, 'parent': 4}, 5: {'start': 3, 'end': 5, 'parent': 5}, 6: {'start': 5, 'end': 6, 'parent': 6}, 7: {'start': 6, 'end': 4, 'parent': 7}, 8: {'start': 5, 'end': 7, 'parent': 8}, 9: {'start': 7, 'end': 8, 'parent': 9}, 10: {'start': 8, 'end': 6, 'parent': 10}, 11: {'start': 7, 'end': 2, 'parent': 11}, 12: {'start': 1, 'end': 8, 'parent': 12}}
{"{'start': 1, 'end': 2, 'parent': 1}": 1, "{'start': 2, 'end': 3, 'parent': 2}": 2, "{'start': 3, 'end': 4, 'parent': 3}": 3, "{'start': 4, 'end': 1, 'parent': 4}": 4, "{'start': 3, 'end': 5, 'parent': 5}": 5, "{'start': 5, 'end': 6, 'parent': 6}": 6, "{'start': 6, 'end': 4, 'parent': 7}": 7, "{'start': 5, 'end': 7, 'parent': 8}": 8, "{'start': 7, 'end': 8, 'parent': 9}": 9, "{'start': 8, 'end': 6, 'parent': 10}": 10, "{'start': 7, 'end': 2, 'parent': 11}": 11, "{'start': 1, 'end': 8, 'parent': 12}": 12}


In [11]:
def quads_vertices():
    res = []
    
    for vx in vertices:
        res.append([ vertices[ vx ][ 0 ], vertices[ vx ][ 1 ], vertices[ vx ][ 2 ]])
    
    return res

mesh_vertices = quads_vertices()

print( mesh_vertices )

[[1.0, 0, 0], [0, 0, 0], [0, 1.0, 0], [1.0, 1.0, 0], [0, 1.0, 1.0], [1.0, 1.0, 1.0], [0, 0, 1.0], [1.0, 0, 1.0]]


In [46]:
def edges_GetTri( edge1, edge2 ):
    vxs = [ -1, -1, -1, -1 ]

    # quads keep edges on a one-based array because negative indicates
    # that we have to consider the vertices in reverse order in that edge.

    id = edge1 if edge1 > 0 else -1 * edge1
    vxs[ 0 ] = edges[ id ][ "start" ] if edge1 > 0 else edges[ id ][ "end" ]
    vxs[ 1 ] = edges[ id ][ "end" ] if edge1 > 0 else edges[id][ "start" ]

    id = edge2 if edge2 > 0 else -1 * edge2
    vxs[ 2 ] = edges[ id ][ "start" ] if edge2 > 0 else edges[ id ][ "end" ]
    vxs[ 3 ] = edges[ id ][ "end" ] if edge2 > 0 else edges[ id ][ "start" ];

    # edges must share a vertex and be informed in correct order
    if vxs[ 1 ] != vxs[ 2 ]:
        return []
    else:
        # one based to zero based
        return [ vxs[ 0 ] - 1, vxs[ 1 ] - 1, vxs[ 3 ] - 1 ]


def quads_GetTris():
    res = []
    for i in range( 0, len( quads )):
        quad = quads_rev[ i ]

        res.append( edges_GetTri( quad[ 0 ][ 0 ], quad[ 0 ][ 1 ] )) #quad.t1.e1, quad.t1.e2 ))
        res.append( edges_GetTri( quad[ 1 ][ 0 ], quad[ 1 ][ 1 ] )) #quad.t2.e3, quad.t2.e4 ))

    return sum( res, []) #.SelectMany( t => new[] { t }).ToArray().SelectMany(x => x).ToArray();

    
mesh_triangles = quads_GetTris()

print( mesh_triangles )

mesh_vertices = [( vx[ 0 ], vx[ 1 ], vx[ 2 ]) for vx in mesh_vertices ]
print( mesh_vertices )


[0, 1, 2, 2, 3, 0, 3, 2, 4, 4, 5, 3, 5, 4, 6, 6, 7, 5, 7, 6, 1, 1, 0, 7, 1, 6, 4, 4, 2, 1, 7, 0, 3, 3, 5, 7]
[(1.0, 0, 0), (0, 0, 0), (0, 1.0, 0), (1.0, 1.0, 0), (0, 1.0, 1.0), (1.0, 1.0, 1.0), (0, 0, 1.0), (1.0, 0, 1.0)]


In [53]:
def vertices_Get( parent_edge, t, pos ):
    key = { "edge": parent_edge, "t": t, "pos": pos }

    if str( key ) in vertex_in_edge:
        return vertex_in_edge[ str( key )]
    else:
        return 0


In [60]:
def quads_GetShared( edgeid ):
    if edgeid in edges_quads:
        return edges_quads[ edgeid ]
    else:
        return ( -1, -1 )

    

In [59]:
def vertices_AddDemi( vertex, parent_edge, t, pos ):
    if parent_edge == 0:
        print( "Vertices must always belong to edges ( parent_edge cannot be zero )" )
        return None

    if t <= 0:
        print( "t must be positive and greater than zero (num of divisions of an edge)" )
        return None

    if pos < 0 or pos > t:
        print( "pos must be in [ 0, t ]" )
        return None

    # check if a vx at that position already exists, and promote it
    # to real vx if it is not one already and all edges sharing it are real edges
    vx_key = { "edge": parent_edge, "t": t, "pos": pos }

    _exists = vertices_Get( vx_key )
    ikey = _exists if _exists > 0 else len( vertices ) + 1

    demi = 3 
    
    if _exists > 0:    
        # the vertice exists, chech if it can be promoted 
        # The vc can be promoted if it belongs to a demi edge and we try to create
        # the same divisions on the second quad that edge shares.
        if vertices[ ikey ][ demi ] == True:
            quads = quads_GetShared( parent_edge )

            if quads[ 1 ] == -1:
                vx = vertices[ ikey ]
                vertices[ ikey ] = ( vx[ 0 ], vx[ 1 ], vx[ 2 ], False )

        return ikey

    # if the vx is new, add it as a demivertice (it is part of a demi edge)
    vc = vertex

    vertices[ ikey ] = ( vc[ 0 ], vc[ 1 ], vc[ 2 ], True )
    parent_edges[ ikey ] = [ vx_key ]

    # a new demiedge is always added when a new demivc is
    vertex_in_edge[ str( vx_key )] = ikey

    return -1 * ikey


In [72]:
def vertices_AddParent( vx, edge, t, pos ):
    vx_key = { "edge": edge, "t": t, "pos": pos }
    
    if  parent_edges[ vx ].count( vx_key ) == 0:
        parent_edges[ vx ].append( vx_key )

    if str( vx_key ) not in vertex_in_edge:
        vertex_in_edge[str( vx_key )] = vx


In [71]:
def edges_Add( start, end, parent ):
    # vcs always > 0
    if start <= 0 or end <= 0:
        print( "indices must be greater than zero" )
        return

    # vertices must exist
    if start > len( vertices ) or end > len( vertices ):
        print( "invalid index (inexistent vertex)" )
        return

    # a demi edge is not always a child of another, but a hard edge
    # always declare itself as a parent
    demi = 3
    _demi = False
    
    if vertices[ start ][ demi ] == True or vertices[ end ][ demi ] == True:
        _demi = True

    if _demi == False and parent != ( len( edges ) + 1 ):
        print( "hard edges cannot have parents" )
        return

    _edge = { "start": start, "end": end, "parent": parent }
    
    if _demi == True and parent == ( len( edges ) + 1 ):
        edges[ len( edges ) + 1 ] =  { "start": start, "end": end, "parent": -1 * parent }
        return

    # TODO: pq abaixo acrescentamos o edges_rev mas no if acima nao?
    edges[ len( edges ) + 1 ] =  _edge
    edges_rev[ str( _edge )] = len( edges )


In [ ]:
def edges_Subdivide( edgeid, slices ):
    start = edges[ edgeid ][ "start" ]
    end = edges[ edgeid ][ "end" ]

    vx_start = vertices[ start ]
    vx_end = vertices[ end ]

    # create a list with intermediate vertices (generate ones if they don´t exist,
    # promote to real vx if they exist
    vxs = [ start ]

    q = vector( vx_end[ 0 ], vx_end[ 1 ], vx_end[ 2 ]) - vector( vx_start[ 0 ], vx_start[ 1 ], vx_start[ 2 ])
    
    print( q )

    # for pos in 1 to slices - 1:
    # add the next vertice as demivertice
    # (vxs deal with their own promotion to real verticies if needed)
    # add each edge start_vx -> prox (then start = prox)
    # at the ent add edge prox -> end_vx
    t = slices
    next = start
    exists = True

    for pos in range( 1, slices ):
    
        prox = vertices_Get( edgeid, t, pos )

        if prox == 0:
            exists = False
            v3_prox = vector( vx_start[ 0 ], vx_start[ 1 ], vx_start[ 2 ]) + q / t * pos;

            # this function promotes a demivx to a real vx if needed
            prox = vertices_AddDemi( v3_prox, edgeid, t, pos )
            if prox < 0:
                prox = -1 * prox

            # declare the parenting of the edge to the vertices
            vertices_AddParent( next, len( edges ) + 1, 1, 0 )
            vertices_AddParent( prox, len( edges ) + 1, 1, 1 )

        edges_Add( next, prox, edgeid )
        next = prox

    edges_Add( next, end, edgeid )

    if exists == False:
        vertices_AddParent( next, len( edges ), 1, 0 )
        vertices_AddParent( end, len( edges ), 1, 1  )


In [41]:
def Select():
    # quads, unlike edges and vertices, is a zero based array
    return [ id for id in range( len( quads ))]

    
def Subdivide( selection, divisions = "2" ):
    
    # if selected other than quads, select all quads
    if selection[ "type" ] != "quads":
        selection = { "type": "quads", "items": Select()}
                     
    _divisions = int( divisions )

    # subdivide selected quads
    for i in range( 0, len( selection[ items ])):
        quad = quads_rev[ selection[ "items" ][ i ]]

        # 1 - subdivide all edges
        edges_Subdivide( quad.t1.e1, _divisions );
        edges_Subdivide( quad.t1.e2, _divisions );
        edges_Subdivide( quad.t2.e3, _divisions );
        edges_Subdivide( quad.t2.e4, _divisions );


Subdivide({ "type": None }, 3 )
                     

NameError: name 'items' is not defined

In [32]:
from vpython import *

ctr = vec( 150, 50, 150 )

# This is needed in Jupyter notebook and lab to make programs easily rerunnable
scene = canvas( center=ctr, background = color.gray( 0.8 ))
scene.autoscaling = False
scene.forward = vec( 0, 2, 0 )
scene.up = vec( 0, 0, 1 )
scene.camera.pos = vec( 1.5, -3, 1.5 )
scene.range = 0.6

verts = []

for vx in mesh_vertices:
    _pos = vector( vx[ 0 ], vx[ 1 ], vx[ 2 ])
    verts.append( vertex( pos = _pos, color = _pos ))

tris = []

for i in range( 0, len( mesh_triangles ), 3 ):
    _v0 = verts[ mesh_triangles[ i ]]
    _v1 = verts[ mesh_triangles[ i + 1 ]]
    _v2 = verts[ mesh_triangles[ i + 2 ]]
    
    
    tris.append( triangle( v0 = _v0, v1 = _v1, v2 = _v2 ))


<IPython.core.display.Javascript object>